In [7]:
import pandas as pd
import numpy as np

data_dir = "Datasets"
continent_dir = "Asia"

df = pd.read_csv(f"../{data_dir}/BioTIMEQuery_24_06_2021.csv")

/Users/ethankaji/Development/CSDS_446/venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3577: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
lat_min = -10
lat_max = 80
long_min = 60
long_max = 157
continent_data = df[df["LATITUDE"].between(lat_min, lat_max) & df["LONGITUDE"].between(long_min, long_max)]

In [9]:
continent_data.to_csv(f"../{data_dir}/{continent_dir}/full.csv")

continent_data

,Unnamed: 0,STUDY_ID,DAY,MONTH,YEAR,SAMPLE_DESC,PLOT,ID_SPECIES,LATITUDE,LONGITUDE,sum.allrawdata.ABUNDANCE,sum.allrawdata.BIOMASS,GENUS,SPECIES,GENUS_SPECIES
293245,293549,96,10.0,9.0,2000,71.966700_82.616700_Arctic_Kara_Sea_Nansen_net...,NaN,12898,71.9667,82.6167,67.373400,0.0,Copepoda,NaN,Copepoda
293246,293550,96,10.0,9.0,2000,71.966700_82.616700_Arctic_Kara_Sea_Nansen_net...,NaN,26805,71.9667,82.6167,0.696136,0.0,Bosmina,longirostris,Bosmina longirostris
293247,293551,96,10.0,9.0,2000,71.966700_82.616700_Arctic_Kara_Sea_Nansen_net...,NaN,27038,71.9667,82.6167,11.970400,0.0,Calanus,glacialis,Calanus glacialis
293248,293552,96,10.0,9.0,2000,71.966700_82.616700_Arctic_Kara_Sea_Nansen_net...,NaN,28261,71.9667,82.6167,74.613200,0.0,Cyclops,strenuus,Cyclops strenuus
293249,293553,96,10.0,9.0,2000,71.966700_82.616700_Arctic_Kara_Sea_Nansen_net...,NaN,28370,71.9667,82.6167,24.817300,0.0,Daphnia,NaN,Daphnia
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8435063,8456245,510,NaN,8.0,2013,37.461_128.53_8_2013_7,7,50911,37.4610,128.5300,3.000000,0.0,Argynnis,niobe,Argynnis niobe
8435064,8456246,510,NaN,8.0,2013,37.461_128.53_8_2013_7,7,50966,37.4610,128.5300,4.000000,0.0,Minois,dryas,Minois dryas
8435065,8456247,510,NaN,8.0,2013,37.461_128.53_8_2013_7,7,50952,37.4610,128.5300,4.000000,0.0,Lethe,marginalis,Lethe marginalis
8435066,8456248,510,NaN,8.0,2013,37.461_128.53_8_2013_7,7,50953,37.4610,128.5300,1.000000,0.0,Libythea,lepita,Libythea lepita


In [10]:
studies = np.unique(continent_data["STUDY_ID"])

In [11]:
for study in studies:
    study_data = continent_data[continent_data["STUDY_ID"] == study]
    print(f"Study ID: {study}")
    print(f"\tYears: {np.unique(study_data['YEAR'])}")
    print(f"\tSpecies Count: {len(np.unique(study_data['GENUS_SPECIES']))}")
    print(f"\tData Points per Species: {len(study_data) / len(np.unique(study_data['GENUS_SPECIES']))}")

SyntaxError: f-string: unmatched '[' (1816798669.py, line 4)